### Imports & basic config

In [18]:
# Core
import re, numpy as np, pandas as pd
from math import sin, cos, radians, asin, sqrt

# Modeling
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score, average_precision_score
from xgboost import XGBClassifier

# Early stopping (newer XGBoost). Safe to ignore if not available.
try:
    from xgboost.callback import EarlyStopping as XGB_EarlyStopping
except Exception:
    XGB_EarlyStopping = None


### Data Injest

In [19]:
ev = pd.read_csv("/home/britton/Projects/Predictive_Analysis_1/data/Electric_Vehicle_Population_Data.csv")
cs = pd.read_csv("/home/britton/Projects/Predictive_Analysis_1/data/charging_stations.csv")

### H3 helpers (v3/v4 compatible)

In [20]:
# Version-agnostic wrappers over h3 (v4 preferred, v3 fallback)
try:
    import h3
    def to_cell(lat, lon, res): return h3.latlng_to_cell(lat, lon, res)
    def cell_center(h): return h3.cell_to_latlng(h)        # (lat, lon)
    def parent(h, res): return h3.cell_to_parent(h, res)
    def neighbors(h, k): return h3.grid_disk(h, k)         # includes center
except Exception:
    from h3 import h3 as h3v3
    def to_cell(lat, lon, res): return h3v3.geo_to_h3(lat, lon, res)
    def cell_center(h): return h3v3.h3_to_geo(h)           # (lat, lon)
    def parent(h, res): return h3v3.h3_to_parent(h, res)
    def neighbors(h, k): return h3v3.k_ring(h, k)          # includes center


### Helpers: WKT parser + Haversine

In [21]:
def parse_point_wkt(s):
    """
    Parse 'POINT(lon lat)' from a WKT-like string.
    Returns (lat, lon); returns (nan, nan) on failure/NaN.
    """
    if pd.isna(s): return np.nan, np.nan
    m = re.search(r"POINT\s*\(\s*([-\d\.]+)\s+([-\d\.]+)\s*\)", str(s))
    if not m: return np.nan, np.nan
    lon, lat = float(m.group(1)), float(m.group(2))
    return lat, lon

def hav_km(lat1, lon1, lat2, lon2):
    R=6371.0
    dlat, dlon = radians(lat2-lat1), radians(lon2-lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1))*cos(radians(lat2))*sin(dlon/2)**2
    return 2*R*asin(sqrt(a))


### Input DataFrames (ev, cs) + basic cleaning

In [22]:
# Expect ev, cs already in memory.

ev = ev.copy()

# Ensure lat/lon exist (either via WKT or Latitude/Longitude)
if "lat" not in ev.columns or "lon" not in ev.columns:
    if "Vehicle Location" in ev.columns:
        ev["lat"], ev["lon"] = zip(*ev["Vehicle Location"].map(parse_point_wkt))
    else:
        ev = ev.rename(columns={"Latitude":"lat","Longitude":"lon"})

# Filter to WA if available, coerce coords, drop bad rows
if "State" in ev.columns:
    ev = ev[ev["State"].astype(str).str.upper().eq("WA")]
ev["lat"] = pd.to_numeric(ev["lat"], errors="coerce")
ev["lon"] = pd.to_numeric(ev["lon"], errors="coerce")
ev = ev.dropna(subset=["lat","lon"]).copy()

# Stations table (cs -> st), filter and normalize schema
st = cs.copy()
if "Fuel Type Code" in st.columns:
    st = st[st["Fuel Type Code"].astype(str).str.upper().eq("ELEC")]
if "State" in st.columns:
    st = st[st["State"].astype(str).str.upper().eq("WA")]
st = st.rename(columns={
    "Latitude":"lat","Longitude":"lon",
    "EV DC Fast Count":"dcfc_count",
    "EV Level2 EVSE Num":"l2_count"
})
st["lat"] = pd.to_numeric(st["lat"], errors="coerce")
st["lon"] = pd.to_numeric(st["lon"], errors="coerce")
for c in ["dcfc_count","l2_count"]:
    if c in st.columns:
        st[c] = pd.to_numeric(st[c], errors="coerce").fillna(0).astype(int)
    else:
        st[c] = 0
st = st.dropna(subset=["lat","lon"]).copy()


### H3 assignment + per-hex aggregates

In [23]:
RES = 8  # neighborhood-ish scale

# Assign EVs & stations to hexes
ev["h3"] = [to_cell(r.lat, r.lon, RES) for r in ev.itertuples()]
st["h3"] = [to_cell(r.lat, r.lon, RES) for r in st.itertuples()]

# Per-hex aggregates
ev_cnt = ev.groupby("h3").size().rename("ev_cnt")
st_cnt = st.groupby("h3").size().rename("station_cnt")
dcfc_cnt = st.groupby("h3")["dcfc_count"].sum().rename("dcfc_ports")
l2_cnt   = st.groupby("h3")["l2_count"].sum().rename("l2_ports")

# Join and label (1 if station exists in hex)
df = pd.concat([ev_cnt, st_cnt, dcfc_cnt, l2_cnt], axis=1).fillna(0)
df["label"] = (df["station_cnt"] > 0).astype(int)

# Hex centers for distance + map
centers = pd.DataFrame({h: cell_center(h) for h in df.index}).T.rename(columns={0:"lat_c",1:"lon_c"})
df = df.join(centers)


### Neighborhood (k=2) features

In [24]:
def ksum(series_map, h, k=2):
    ns = set(neighbors(h, k))
    ns.discard(h)                 # drop self
    return sum(series_map.get(n, 0) for n in ns)


# Precompute dicts for fast lookup
ev_map, st_map, dc_map, l2_map = ev_cnt.to_dict(), st_cnt.to_dict(), dcfc_cnt.to_dict(), l2_cnt.to_dict()

# k=2 totals (context)
df["ev_cnt_k2"]     = [ksum(ev_map, h, 2) for h in df.index]
df["st_cnt_k2"]     = [ksum(st_map, h, 2) for h in df.index]
df["dcfc_ports_k2"] = [ksum(dc_map, h, 2) for h in df.index]
df["l2_ports_k2"]   = [ksum(l2_map, h, 2) for h in df.index]


### Distance-to-nearest-station feature

In [25]:
# Build array of station hex centers
station_hexes = st["h3"].unique().tolist()
station_centers = np.array([cell_center(h) for h in station_hexes])

def nearest_station_km(lat, lon):
    """Brute-force nearest station distance (km)."""
    if station_centers.size == 0: return np.nan
    return float(np.min([hav_km(lat, lon, slat, slon) for slat, slon in station_centers]))

df["dist_to_station_km"] = [nearest_station_km(r.lat_c, r.lon_c) for r in df.itertuples()]
df["dist_to_station_km"] = df["dist_to_station_km"].replace([np.inf,-np.inf], np.nan)
df["dist_to_station_km"] = df["dist_to_station_km"].fillna(df["dist_to_station_km"].max() or 0)


### Spatial CV training & metrics

In [26]:
FEATURES = ["ev_cnt","ev_cnt_k2","st_cnt_k2","dcfc_ports_k2","l2_ports_k2","dist_to_station_km"]
X, y = df[FEATURES].copy(), df["label"].astype(int)

# Group by parent hexes at a coarser res to reduce leakage across folds
GROUP_RES = 5
groups = [parent(h, GROUP_RES) for h in df.index]
gkf = GroupKFold(n_splits=5)

def fit_xgb(model, Xtr, ytr, Xva, yva):
    """Try modern callbacks -> early_stopping_rounds -> no ES."""
    if XGB_EarlyStopping is not None:
        try:
            return model.fit(
                Xtr, ytr, eval_set=[(Xva, yva)], verbose=False,
                callbacks=[XGB_EarlyStopping(rounds=50, save_best=True)]
            )
        except TypeError:
            pass
    try:
        return model.fit(
            Xtr, ytr, eval_set=[(Xva, yva)], verbose=False,
            early_stopping_rounds=50
        )
    except TypeError:
        return model.fit(Xtr, ytr, eval_set=[(Xva, yva)], verbose=False)

aucs, auprcs = [], []
for tr, va in gkf.split(X, y, groups):
    Xtr, Xva = X.iloc[tr], X.iloc[va]
    ytr, yva = y.iloc[tr], y.iloc[va]
    pos = max(ytr.mean(), 1e-8)
    spw = (1 - pos) / pos

    model = XGBClassifier(
        n_estimators=600, learning_rate=0.05, max_depth=6,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=2.0,
        objective="binary:logistic", eval_metric="logloss",
        tree_method="hist", random_state=42, scale_pos_weight=spw
    )
    fit_xgb(model, Xtr, ytr, Xva, yva)
    p = model.predict_proba(Xva)[:, 1]  # <-- P(station_present) on validation fold
    aucs.append(roc_auc_score(yva, p))
    auprcs.append(average_precision_score(yva, p))

print(f"Spatial CV AUROC: {np.mean(aucs):.3f} ± {np.std(aucs):.3f}")
print(f"Spatial CV AUPRC: {np.mean(auprcs):.3f} ± {np.std(auprcs):.3f}")


Spatial CV AUROC: 0.999 ± 0.001
Spatial CV AUPRC: 1.000 ± 0.000


### Final fit on all data + per-hex score

In [27]:
pos = max(y.mean(), 1e-8)
spw = (1 - pos) / pos

final_model = XGBClassifier(
    n_estimators=600, learning_rate=0.05, max_depth=6,
    subsample=0.8, colsample_bytree=0.8, reg_lambda=2.0,
    objective="binary:logistic", eval_metric="logloss",
    tree_method="hist", random_state=42, scale_pos_weight=spw
)
final_model.fit(X, y, verbose=False)

# <-- Prediction for every hex: P(station_present)
df["score"] = final_model.predict_proba(X)[:, 1]


### Candidate selection & CSV export

In [28]:
# Keep consistent hex id column on export
df = df.copy()
df.index.name = "h3"

# Policy: no current station AND >= 2 km from nearest station
candidates = df[df["label"] == 0].copy()
candidates = candidates[candidates["dist_to_station_km"] >= 2.0]

out = candidates.sort_values("score", ascending=False).reset_index()
if "h3" not in out.columns and "index" in out.columns:
    out = out.rename(columns={"index": "h3"})
out = out.rename(columns={"h3": "h3_cell"})

cols = [
    "h3_cell","lat_c","lon_c",
    "ev_cnt","ev_cnt_k2",
    "st_cnt_k2","dcfc_ports_k2","l2_ports_k2",
    "dist_to_station_km","score"
]

missing = [c for c in cols if c not in out.columns]
if missing:
    raise KeyError(f"Missing columns in export: {missing}. Present: {list(out.columns)}")

out[cols].to_csv("/home/britton/Projects/Predictive_Analysis_1/data/charging_site_candidates.csv", index=False)
print("Wrote charging_site_candidates.csv")


Wrote charging_site_candidates.csv


### (Optional) Feature importances

In [29]:
feat_names = list(X.columns)
booster = final_model.get_booster()
name_map = {f"f{i}": feat_names[i] for i in range(len(feat_names))}

def imp_df(importance_type="gain"):
    raw = booster.get_score(importance_type=importance_type)
    if not raw:
        return pd.Series(0.0, index=feat_names, name=f"xgb_{importance_type}")
    # If keys look like 'f0', map them; if keys are real names, use as-is
    if set(raw.keys()) <= set(name_map.keys()):
        s = pd.Series({name_map[k]: v for k, v in raw.items()})
    else:
        s = pd.Series(raw)
    s = s.reindex(feat_names).fillna(0.0).sort_values(ascending=False)
    s.name = f"xgb_{importance_type}"
    return s

gain_imp   = imp_df("gain")
cover_imp  = imp_df("cover")
weight_imp = imp_df("weight")
pd.concat([gain_imp, cover_imp, weight_imp], axis=1)



,xgb_gain,xgb_cover,xgb_weight
dist_to_station_km,53.092361,32.850357,136.0
ev_cnt_k2,14.531989,54.808544,19.0
ev_cnt,9.291797,9.821816,241.0
l2_ports_k2,0.907876,9.897997,137.0
st_cnt_k2,0.605632,11.132406,106.0
dcfc_ports_k2,0.190432,6.937555,126.0


### Folium map (load CSV and render)

In [ ]:
import folium
from folium import LayerControl
from branca.colormap import linear

CAND_PATH = "/home/britton/Projects/Predictive_Analysis_1/data/charging_site_candidates.csv"
TOP_N = 300

cand = pd.read_csv(CAND_PATH)
if "h3_cell" not in cand.columns:
    if "h3" in cand.columns:
        cand = cand.rename(columns={"h3": "h3_cell"})
    elif "index" in cand.columns:
        cand = cand.rename(columns={"index": "h3_cell"})

required_cols = {"h3_cell", "lat_c", "lon_c", "score"}
missing = required_cols - set(cand.columns)
if missing:
    raise ValueError(f"Candidates CSV missing required columns: {missing}")

cand = cand.dropna(subset=["h3_cell", "lat_c", "lon_c", "score"]).copy()
smin, smax = cand["score"].min(), cand["score"].max()
cand["score_norm"] = (cand["score"] - smin) / (smax - smin + 1e-9)

# H3 boundary helper for map
try:
    import h3  # v4+
    def hex_boundary(h): return h3.cell_to_boundary(h)          # [(lat, lon), ...]
except Exception:
    from h3 import h3 as h3v3
    def hex_boundary(h): return h3v3.h3_to_geo_boundary(h)      # [(lat, lon), ...]

# Station points (from in-memory cs)
st_map = cs.copy().rename(columns={"Latitude":"lat","Longitude":"lon"}).dropna(subset=["lat","lon"]).copy()

# Base map
m = folium.Map(location=[47.5, -120.5], zoom_start=6, tiles="cartodbpositron")
cmap = linear.YlOrRd_09.scale(0, 1); cmap.caption = "Candidate score (higher = more promising)"; cmap.add_to(m)

# Top-N polygons with tooltips
top_layer = folium.FeatureGroup(name=f"Top {TOP_N} candidates", show=True)
for r in cand.nlargest(TOP_N, "score").itertuples():
    try:
        poly = hex_boundary(r.h3_cell)
        folium.Polygon(
            locations=poly,
            tooltip=(f"<b>Score:</b> {r.score:.3f}<br>"
                     f"<b>EV in hex:</b> {int(getattr(r, 'ev_cnt', 0))} | "
                     f"<b>EV k2:</b> {int(getattr(r, 'ev_cnt_k2', 0))}<br>"
                     f"<b>Stations k2:</b> {int(getattr(r, 'st_cnt_k2', 0))} | "
                     f"<b>DCFC k2:</b> {int(getattr(r, 'dcfc_ports_k2', 0))} | "
                     f"<b>L2 k2:</b> {int(getattr(r, 'l2_ports_k2', 0))}<br>"
                     f"<b>Dist to station:</b> {float(getattr(r, 'dist_to_station_km', np.nan)):.1f} km"),
            weight=1, opacity=0.8, fill_opacity=0.55, color="#333333",
            fill_color=cmap(r.score_norm)
        ).add_to(top_layer)
    except Exception:
        continue
top_layer.add_to(m)

# All candidates (heavier)
all_layer = folium.FeatureGroup(name="All candidates", show=False)
for r in cand.itertuples():
    try:
        poly = hex_boundary(r.h3_cell)
        folium.Polygon(
            locations=poly, weight=0.3, opacity=0.5, fill_opacity=0.35,
            color="#555555", fill_color=cmap(r.score_norm)
        ).add_to(all_layer)
    except Exception:
        continue
all_layer.add_to(m)

# Existing stations
st_layer = folium.FeatureGroup(name="Existing stations", show=True)
for r in st_map.itertuples():
    try:
        folium.CircleMarker(
            location=(float(r.lat), float(r.lon)),
            radius=2.5, fill=True, fill_opacity=0.8, opacity=0.8
        ).add_to(st_layer)
    except Exception:
        continue
st_layer.add_to(m)

LayerControl(collapsed=False).add_to(m)
m.save("/home/britton/Projects/Predictive_Analysis_1/data/wa_ev_charging_candidates_map.html")
print("Wrote wa_ev_charging_candidates_map.html")


Wrote wa_ev_charging_candidates_map.html
